<a href="https://colab.research.google.com/github/miwytt/atmt_2022/blob/assignment05/mwastl_atmt_assignment05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment preparation



In [2]:
from google.colab import drive
import torch

In [5]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-rr56x_9z
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-rr56x_9z
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 118 kB 38.0 MB/s 


In [6]:
# mounting drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [7]:
# check for GPU(s)
if torch.cuda.is_available():       
    cuda = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    cuda = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [8]:
!pwd

/content


In [9]:
%cd drive/MyDrive/atmt/assignment05
%ls

/content/drive/MyDrive/atmt/assignment05
 atmt_2022/
'Bildschirmfoto 2022-12-11 um 08.48.28.png'
 cding_mwastl_atmt_assignment05_Q3_2.gsheet
 cding_mwastl_atmt_assignment05_Q3.gsheet
 mwastl_atmt_assignment05.ipynb


In [10]:
# cloning the repository 
!git clone --branch assignment05 https://github.com/miwytt/atmt_2022/
!git checkout assignment05
!git branch

fatal: destination path 'atmt_2022' already exists and is not an empty directory.
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [11]:
%cd atmt_2022/
%ls

/content/drive/MyDrive/atmt/assignment05/atmt_2022
assignments/  moses_scripts/  README.md         seq2seq/  translate_beam.py
data/         preprocess.py   requirements.txt  share/    translate.py
LICENSE       __pycache__/    scripts/          train.py


# 1 Experimenting with Beam Search

## Experimenting with Beam Sizes

Beam sizes: 1 <= k <= 25.

Baseline model: assignments/03/baseline/checkpoints (hallerp version)

In [ ]:
for i in range(1, 26):
  # run inference on test set
  !python translate_beam.py \
      --data data/en-fr/prepared \
      --dicts data/en-fr/prepared \
      --checkpoint-path assignments/03/baseline/checkpoints/checkpoint_last.pt \
      --output assignments/05/squared_regularizer/translations.txt \
      --cuda True \
      --beam-size {i}

  # Postprocess model translations
  !bash scripts/postprocess.sh \
      assignments/05/beam_sizes/translations.txt \
      assignments/05/beam_sizes/translations.p.txt en

  # Score with SacreBLEU
  !cat assignments/05/beam_sizes/translations.p.txt | sacrebleu data/en-fr/raw/test.en


# 4 Investigating the Diversity of Beam Search

In [19]:
for g in [0.5, 1, 1.5]:
  # run inference on test set
  !python translate_beam.py \
      --data data/en-fr/prepared \
      --dicts data/en-fr/prepared \
      --checkpoint-path assignments/03/baseline/checkpoints/checkpoint_last.pt \
      --output assignments/05/diversity/translations_div_{g}.txt \
      --cuda True \
      --n 3 \
      --gamma {g} \


  # Postprocess model translations
  !bash scripts/postprocess.sh \
      assignments/05/diversity/translations_div_{g}.txt \
      assignments/05/diversity/translations_div_{g}.p.txt en

[2022-12-13 09:11:10] COMMAND: translate_beam.py --data data/en-fr/prepared --dicts data/en-fr/prepared --checkpoint-path assignments/03/baseline/checkpoints/checkpoint_last.pt --output assignments/05/diversity/translations_div_0.5.txt --cuda True --n 3 --gamma 0.5
[2022-12-13 09:11:10] Arguments: {'cuda': 'True', 'data': 'data/en-fr/prepared', 'source_lang': 'fr', 'target_lang': 'en', 'max_tokens': None, 'batch_size': None, 'train_on_tiny': False, 'arch': 'lstm', 'max_epoch': 10000, 'clip_norm': 4.0, 'lr': 0.0003, 'patience': 3, 'log_file': None, 'save_dir': '/home/user/kew/HS2021/atmt/tests/assignment_03/baseline/checkpoints', 'restore_file': 'checkpoint_last.pt', 'save_interval': 1, 'no_save': False, 'epoch_checkpoints': False, 'encoder_embed_dim': 64, 'encoder_embed_path': None, 'encoder_hidden_size': 64, 'encoder_num_layers': 1, 'encoder_bidirectional': 'True', 'encoder_dropout_in': 0.25, 'encoder_dropout_out': 0.25, 'decoder_embed_dim': 64, 'decoder_embed_path': None, 'decoder_hi

In [18]:
# run inference on test set
!python translate_beam.py \
    --data data/en-fr/prepared \
    --dicts data/en-fr/prepared \
    --checkpoint-path assignments/03/baseline/checkpoints/checkpoint_last.pt \
    --output assignments/05/diversity/translations_standard.txt \
    --cuda True \
    --n 3 \


# Postprocess model translations
!bash scripts/postprocess.sh \
    assignments/05/diversity/translations_standard.txt \
    assignments/05/diversity/translations_standard.p.txt en

[2022-12-13 09:09:22] COMMAND: translate_beam.py --data data/en-fr/prepared --dicts data/en-fr/prepared --checkpoint-path assignments/03/baseline/checkpoints/checkpoint_last.pt --output assignments/05/diversity/translations_standard.txt --cuda True --n 3
[2022-12-13 09:09:22] Arguments: {'cuda': 'True', 'data': 'data/en-fr/prepared', 'source_lang': 'fr', 'target_lang': 'en', 'max_tokens': None, 'batch_size': None, 'train_on_tiny': False, 'arch': 'lstm', 'max_epoch': 10000, 'clip_norm': 4.0, 'lr': 0.0003, 'patience': 3, 'log_file': None, 'save_dir': '/home/user/kew/HS2021/atmt/tests/assignment_03/baseline/checkpoints', 'restore_file': 'checkpoint_last.pt', 'save_interval': 1, 'no_save': False, 'epoch_checkpoints': False, 'encoder_embed_dim': 64, 'encoder_embed_path': None, 'encoder_hidden_size': 64, 'encoder_num_layers': 1, 'encoder_bidirectional': 'True', 'encoder_dropout_in': 0.25, 'encoder_dropout_out': 0.25, 'decoder_embed_dim': 64, 'decoder_embed_path': None, 'decoder_hidden_size':

## Push to Repository

In [ ]:
!pwd

/content


In [29]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/miwytt/atmt_2022
   8f952b2..b488d51  assignment05 -> origin/assignment05
Updating 8f952b2..b488d51
Fast-forward
 translate_beam.py | 8 +++++---
 1 file changed, 5 insertions(+), 3 deletions(-)


In [14]:
!git checkout b488d5137d655fdee5623c1ad3b2feac68edea52

M	moses_scripts/detokenizer.perl
M	moses_scripts/detruecase.perl
Note: checking out 'b488d5137d655fdee5623c1ad3b2feac68edea52'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at b488d51 Update translate_beam.py
